# date_time

In [ ]:
open rust_operators
open sm'_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::DateTime<$0>\")>]\n#endif\ntype chrono_DateTime<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Local\")>]\n#endif\ntype chrono_Local = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::NaiveDateTime\")>]\n#endif\ntype chrono_NaiveDateTime = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Utc\")>]\n#endif\ntype chrono_Utc = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::time::Duration\")>]\n#endif\ntype std_time_Duration = class end"

()



In [ ]:
inl types () =
    sm'.types ()
    types ()

()



In [ ]:
nominal timestamp = i64
nominal duration = $"std_time_Duration"
nominal date_time = $"System.DateTime"
nominal date_time' t = $"chrono_DateTime<`t>"
nominal local = $"chrono_Local"
nominal naive_date_time = $"chrono_NaiveDateTime"
nominal utc = $"chrono_Utc"

()



## date_time_milliseconds

In [ ]:
inl date_time_milliseconds
    (year : int) (month : int) (day : int) (hour : int) (minute : int) (second : int) (millisecond : int)
    : date_time
    =
    $'System.DateTime (!year, !month, !day, !hour, !minute, !second, !millisecond)'

()



## date_time_utc

In [ ]:
inl date_time_utc
    (year : int) (month : int) (day : int) (hour : int) (minute : int) (second : int)
    : date_time
    =
    $'System.DateTime (!year, !month, !day, !hour, !minute, !second, System.DateTimeKind.Utc)'

()



## naive_utc

In [ ]:
inl naive_utc (date_time : date_time' utc) : naive_date_time =
    inl date_time = join date_time
    !\($'"!date_time.naive_utc()"')

()



## to_local

In [ ]:
inl to_local (date_time : date_time' utc) : date_time' local =
    inl naive_date_time = date_time |> naive_utc
    !\($'"chrono::offset::TimeZone::from_utc_datetime(&chrono::Local, &!naive_date_time)"')

()



## from_timestamp_micros

In [ ]:
inl from_timestamp_micros forall t {number; int}. (timestamp : t) : option (date_time' utc) =
    inl result : optionm'.option' (date_time' utc) =
        !\\(timestamp, $'"chrono::DateTime::from_timestamp_micros($0)"')
    result |> optionm'.unbox

()



## ticks

In [ ]:
inl ticks (date_time : date_time) : timestamp =
    date_time |> $'_.Ticks'

()



## format

In [ ]:
inl format (format : string) (date_time : date_time) : string =
    $'!date_time.ToString' format

()



## format_iso8601

In [ ]:
inl format_iso8601 (date_time : date_time) =
    date_time |> format "yyyy-MM-ddTHH-mm-ss.fff"

()



## format'

In [ ]:
inl format' (format : string) (date_time : date_time' utc) : sm'.std_string =
    inl format = #format
    inl date_time = join date_time
    !\($'"!date_time.format(!format).to_string()"')

()



## format''

In [ ]:
inl format'' (format : string) (date_time : date_time' _) : sm'.std_string =
    inl format = #format
    inl date_time = join date_time
    !\($'"!date_time.format(!format).to_string()"')

()



## format_timestamp

In [ ]:
inl format_timestamp forall t {number; int}. (timestamp : t) =
    inl timestamp = join timestamp
    (timestamp / 1000)
    |> from_timestamp_micros
    |> optionm.map fun x =>
        x
        |> to_local
        |> format'' "%Y-%m-%d %H:%M:%S"
        |> sm'.from_std_string
    |> resultm.from_option

()



## duration_from_millis

In [ ]:
inl duration_from_millis (ms : u64) : duration =
    inl ms = join ms
    !\($'"std::time::Duration::from_millis(!ms)"')

()



## min_value

In [ ]:
inl min_value () : date_time =
    $'System.DateTime.MinValue'

()



## max_value

In [ ]:
inl max_value () : date_time =
    $'System.DateTime.MaxValue'

()



## unix_epoch

In [ ]:
inl unix_epoch () : date_time =
    $'System.DateTime.UnixEpoch'

()



## to_universal_time

In [ ]:
inl to_universal_time (date_time : date_time) : date_time =
    date_time |> $'_.ToUniversalTime()'

()



## date_time_kind

In [ ]:
union date_time_kind =
    | Unspecified
    | Utc
    | Local

()



## specify_date_kind

In [ ]:
inl specify_date_kind (kind : date_time_kind) (date_time : date_time) : date_time =
    inl kind : $'System.DateTimeKind' =
        match kind with
        | Unspecified => $'System.DateTimeKind.Unspecified'
        | Utc => $'System.DateTimeKind.Utc'
        | Local => $'System.DateTimeKind.Local'
    $'System.DateTime.SpecifyKind (!date_time, !kind)'

()



## time_span

In [ ]:
nominal time_span = $'System.TimeSpan'

inl time_span x : time_span =
    $'`time_span !x '

()



## new_time_span

In [ ]:
inl new_time_span (a : date_time) (b : date_time) : time_span =
    $'!b - !a '

()



## time_span_format

In [ ]:
inl time_span_format (format : string) (time_span : time_span) : string =
    $'!time_span.ToString' format

()



## hours

In [ ]:
inl hours (time_span : time_span) : i32 =
    time_span |> $'_.Hours'

()



## milliseconds

In [ ]:
inl milliseconds (time_span : time_span) : i32 =
    time_span |> $'_.Milliseconds'

()



## minutes

In [ ]:
inl minutes (time_span : time_span) : i32 =
    time_span |> $'_.Minutes'

()



## seconds

In [ ]:
inl seconds (time_span : time_span) : i32 =
    time_span |> $'_.Seconds'

()



## total_seconds

In [ ]:
inl total_seconds (time_span : time_span) : f64 =
    time_span |> $'_.TotalSeconds'

()



## time_zone_info

In [ ]:
nominal time_zone_info = $'System.TimeZoneInfo'

()



## time_zone_local

In [ ]:
inl time_zone_local () : time_zone_info =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.TimeZoneInfo.Local'
        | Rust (Native) => fun () =>
            open rust_operators
            !\($'"0i64.into()"')
        | _ => fun () => null ()

()



## get_utc_offset

In [ ]:
inl get_utc_offset (time_zone_info : time_zone_info) (date_time : date_time) : time_span =
    run_target function
        | Fsharp (Native) => fun () =>
            date_time |> $'_.GetUtcOffset' (time_zone_local ())
        | _ => fun () =>
            time_span ()

()



## add_days

In [ ]:
inl add_days (days : i32) (date_time : date_time) : date_time =
    $'!date_time.AddDays' days

()



## now

In [ ]:
inl now () : date_time =
    $'System.DateTime.Now'

()



## utc_now

In [ ]:
inl utc_now () : date_time =
    $'System.DateTime.UtcNow'

()



## timestamp_guid

In [ ]:
type timestamp_guid = guid.guid

()



## date_time_guid

In [ ]:
type date_time_guid = guid.guid

()



In [ ]:
//// test

inl test_guid () =
    guid.new_guid "FEDCBA98-7654-3210-FEDC-BA9876543210"

()



## date_time_guid_from_date_time

In [ ]:
let date_time_guid_from_date_time (guid : guid.guid) (date_time : date_time) =
    inl parse prefix time_zone : date_time_guid =
        inl guid = guid |> sm'.obj_to_string
        $'`date_time_guid $"{!prefix}{!time_zone}{!guid.[!prefix.Length + !time_zone.Length..]}"'
    run_target function
        | Fsharp (Native) => fun () =>
            inl prefix = date_time |> format (join "yyyyMMdd-HHmm-ssff-ffff-f")
            inl time_zone = date_time |> get_utc_offset (time_zone_local ())
            inl time_zone_signal = (hours time_zone > 0) |> convert_i32
            inl time_zone_value = time_zone |> time_span_format (join "hhmm")
            inl time_zone = $'$"{!time_zone_signal}{!time_zone_value}"' : string
            parse prefix time_zone
        | Rust (Native) => fun () =>
            inl epoch =
                date_time_utc 1970 1 1 0 0 0
                |> to_universal_time
            inl date_time =
                date_time
                |> specify_date_kind Local
                |> to_universal_time
            inl unixticks =
                match date_time |> ticks, epoch |> ticks with
                | timestamp date_time, timestamp epoch => date_time - epoch
            inl prefix =
                unixticks / 10
                |> from_timestamp_micros
                |> optionm.map (
                    to_local
                    >> format'' "%Y%m%d-%H%M-%S%f"
                    >> sm'.from_std_string
                    >> fun s => $'$"{!s.[0..17]}-{!s.[18..21]}-{!s.[22]}"'
                )
                |> optionm'.default_value ""
            inl time_zone = date_time |> get_utc_offset (time_zone_local ())
            inl time_zone_signal = if hours time_zone > 0 then 1u8 else 0
            inl time_zone_value = time_zone |> time_span_format "hh:mm"
            inl time_zone = $'$"{!time_zone_signal}{!time_zone_value.[0..1]}{!time_zone_value.[3..4]}"' : string
            parse prefix time_zone
        | _ => fun () => null ()

()



In [ ]:
//// test
//// print_code=false

types ()
now () |> to_universal_time |> date_time_guid_from_date_time (test_guid ()) |> sm'.obj_to_string
|> console.write_line

20240514-0012-1256-5607-500400543210


In [ ]:
//// test
//// print_code=false
///! rust -d chrono

types ()
inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
now ()
|> to_universal_time
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq' $'$"{!s.[0..29]}{!suffix}"'

assert_eq' / actual: "20240514-0012-1753-0571-000000543210" / expected: "20240514-0012-1753-0571-000000543210"


In [ ]:
//// test
//// print_code=false

types ()
inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
min_value ()
|> specify_date_kind Local
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq' $'$"00010101-0000-0000-0000-0{!s.[25..29]}{!suffix}"'

assert_eq' / actual: "00010101-0000-0000-0000-000400543210" / expected: "00010101-0000-0000-0000-000400543210"


In [ ]:
//// test
//// print_code=false
///! rust -d chrono

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
types ()
min_value ()
|> specify_date_kind Local
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq' $'$"00010101-0000-0000-0000-0{!s.[25..29]}{!suffix}"'

assert_eq' / actual: "00010101-0000-0000-0000-000000543210" / expected: "00010101-0000-0000-0000-000000543210"


In [ ]:
//// test
//// print_code=false

types ()
inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
max_value ()
|> specify_date_kind Utc
|> add_days -1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"99991230-2359-5999-9999-9{!s.[25..29]}{!suffix}"'

assert_eq / actual: "99991230-2359-5999-9999-900400543210" / expected: "99991230-2359-5999-9999-900400543210"


In [ ]:
//// test
//// print_code=false
///! rust -d chrono

types ()
inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
max_value ()
|> specify_date_kind Utc
|> add_days -1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"99991230-2359-5999-9999-0{!s.[25..29]}{!suffix}"'

assert_eq / actual: "99991230-2359-5999-9999-000000543210" / expected: "99991230-2359-5999-9999-000000543210"


In [ ]:
//// test
//// print_code=false

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
unix_epoch ()
|> specify_date_kind Utc
|> add_days 1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"19700102-0000-0000-0000-0{!s.[25..29]}{!suffix}"'

assert_eq / actual: "19700102-0000-0000-0000-000400543210" / expected: "19700102-0000-0000-0000-000400543210"


In [ ]:
//// test
//// print_code=false
///! rust -d chrono

types ()
inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x - 6i32) (am'.End id)
unix_epoch ()
|> specify_date_kind Utc
|> add_days 1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"19700102-0000-0000-0000-0{!s.[25..29]}{!suffix}"'

assert_eq / actual: "19700102-0000-0000-0000-000000543210" / expected: "19700102-0000-0000-0000-000000543210"


## date_time_from_guid

In [ ]:
inl date_time_from_guid (date_time_guid : date_time_guid) =
    inl date_time_guid = date_time_guid |> sm'.obj_to_string
    inl sm'_replace = join sm'.replace
    run_target function
        | Rust _ => fun () =>
            $'System.DateTime.Parse (!date_time_guid.[..24] |> !sm'_replace "-" "")' : date_time
        | _ => fun () => $'System.DateTime.ParseExact (!date_time_guid.[..24] |> !sm'_replace "-" "", "yyyyMMddHHmmssfffffff", null)' : date_time

()



In [ ]:
//// test

date_time_from_guid (guid.new_guid "00010101-0000-0000-0000-0a9876543210")
|> _assert_eq' (min_value ())

let rec closure2 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure1 (v0 : string) (v1 : string) : (string -> string) =
    closure2(v0, v1)
and closure0 () (v0 : string) : (string -> (string -> string)) =
    closure1(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure0()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "00010101-0000-0000-0000-0a9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (string -> (string -> (string -> string))) = method1()
    let v5 : bool = true
    let mutable _v5 : System.DateTime option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v6 : System.DateTime = System.DateTime.Parse (v3.[..24] |> v4 "-" "")
    v6 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v7 : System.

In [ ]:
//// test

date_time_from_guid (guid.new_guid $'$"99991231-2359-5999-9999-9{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' (max_value ())

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and closure3 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure2 (v0 : string) (v1 : string) : (string -> string) =
    closure3(v0, v1)
and closure1 () (v0 : string) : (string -> (string -> string)) =
    closure2(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure1()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"99991231-2359-5999-9999-9{(v0 () |> string).[^10..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : (string -> (string -> (string -> string))) = method1()
    let v6 : bool = true
    let mutable _v6 : System.DateTime option = None 
    
#if FABLE_COMPILER || WASM |

In [ ]:
//// test

date_time_from_guid (guid.new_guid $'$"19700101-0000-0000-0000-0{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' $'System.DateTime.UnixEpoch'

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and closure3 (v0 : string, v1 : string) (v2 : string) : string =
    let v3 : string = v2.Replace (v0, v1)
    v3
and closure2 (v0 : string) (v1 : string) : (string -> string) =
    closure3(v0, v1)
and closure1 () (v0 : string) : (string -> (string -> string)) =
    closure2(v0)
and method1 () : (string -> (string -> (string -> string))) =
    closure1()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"19700101-0000-0000-0000-0{(v0 () |> string).[^10..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : (string -> (string -> (string -> string))) = method1()
    let v6 : bool = true
    let mutable _v6 : System.DateTime option = None 
    
#if FABLE_COMPILER || WASM |

## timestamp_guid_from_timestamp

In [ ]:
inl timestamp_guid_from_timestamp (guid : guid.guid) (timestamp : timestamp) : timestamp_guid =
    inl guid = guid |> sm'.obj_to_string
    inl timestamp = timestamp |> sm'.obj_to_string |> sm'.pad_left 18i32 '0'
    $'`timestamp_guid $"{!timestamp.[0..7]}-{!timestamp.[8..11]}-{!timestamp.[12..15]}-{!timestamp.[16..17]}{!guid.[21..]}"'

()



In [ ]:
//// test

timestamp_guid_from_timestamp (test_guid ()) (timestamp 0i64)
|> _assert_eq' (guid.new_guid "00000000-0000-0000-00dc-ba9876543210")

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (int64 -> string) = _.ToString()
    let v5 : string = v4 0L
    let v6 : string = v5.PadLeft (18, '0')
    let v7 : System.Guid = System.Guid $"{v6.[0..7]}-{v6.[8..11]}-{v6.[12..15]}-{v6.[16..17]}{v3.[21..]}"
    let v8 : string = "00000000-0000-0000-00dc-ba9876543210"
    let v9 : System.Guid = System.Guid v8 
    let v10 : bool = v7 = v9
    let v12 : bool =
        if v10 then
            true
        else
            method1(v10)
    let v13 : string = "assert_eq'"
    let v14 : string = $"{v13} / actual: %A{v7} / expected: %A{v9}"
    System.Console.WriteLine v14
    let v15 : bool = v12 = false
    if v15 then
        failwith<unit> v14
method0()

assert_eq' / actual: 00000000-0000-0000-00dc-ba9876543210 / expected: 00

In [ ]:
//// test

timestamp_guid_from_timestamp (test_guid ()) (timestamp 999999999999999999i64)
|> _assert_eq' (guid.new_guid $'$"99999999-9999-9999-99dc-b{(!test_guid () |> string).[^10..]}"')

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : (int64 -> string) = _.ToString()
    let v5 : string = v4 999999999999999999L
    let v6 : string = v5.PadLeft (18, '0')
    let v7 : System.Guid = System.Guid $"{v6.[0..7]}-{v6.[8..11]}-{v6.[12..15]}-{v6.[16..17]}{v3.[21..]}"
    let v8 : (unit -> System.Guid) = closure0()
    let v9 : string = $"99999999-9999-9999-99dc-b{(v8 () |> string).[^10..]}"
    let v10 : System.Guid = System.Guid v9 
    let v11 : bool = v7 = v10
    let v13 : bool =
        if v11 then
            true
        else
            method1(v11)
    let v14 : string = "assert_eq'"
    let v15 : string = $"{v14}

## timestamp_from_guid

In [ ]:
inl timestamp_from_guid (guid : date_time_guid) : timestamp =
    inl guid = guid |> sm'.obj_to_string
    $'`i64 $"{!guid.[0..7]}{!guid.[9..12]}{!guid.[14..17]}{!guid.[19..20]}"'

()



In [ ]:
//// test

timestamp_from_guid (guid.new_guid "00000000-0000-0000-00dc-ba9876543210")
|> _assert_eq (timestamp 0)

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "00000000-0000-0000-00dc-ba9876543210"
    let v1 : System.Guid = System.Guid v0 
    let v2 : (System.Guid -> string) = _.ToString()
    let v3 : string = v2 v1
    let v4 : int64 = int64 $"{v3.[0..7]}{v3.[9..12]}{v3.[14..17]}{v3.[19..20]}"
    let v5 : bool = v4 = 0L
    let v7 : bool =
        if v5 then
            true
        else
            method1(v5)
    let v8 : string = "assert_eq"
    let v9 : string = $"{v8} / actual: %A{v4} / expected: %A{0L}"
    System.Console.WriteLine v9
    let v10 : bool = v7 = false
    if v10 then
        failwith<unit> v9
method0()

assert_eq / actual: 0L / expected: 0L


In [ ]:
//// test

timestamp_from_guid (guid.new_guid $'$"99999999-9999-9999-99{(!test_guid () |> string).[^14..]}"')
|> _assert_eq (timestamp 999999999999999999)

let rec closure0 () () : System.Guid =
    let v0 : string = "FEDCBA98-7654-3210-FEDC-BA9876543210"
    let v1 : System.Guid = System.Guid v0 
    v1
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> System.Guid) = closure0()
    let v1 : string = $"99999999-9999-9999-99{(v0 () |> string).[^14..]}"
    let v2 : System.Guid = System.Guid v1 
    let v3 : (System.Guid -> string) = _.ToString()
    let v4 : string = v3 v2
    let v5 : int64 = int64 $"{v4.[0..7]}{v4.[9..12]}{v4.[14..17]}{v4.[19..20]}"
    let v6 : bool = v5 = 999999999999999999L
    let v8 : bool =
        if v6 then
            true
        else
            method1(v6)
    let v9 : string = "assert_eq"
    let v10 : string = $"{v9} / actual: %A{v5} / expected: %A{999999999999999999L}"
    System.Console.WriteLine v10
    let v11 : bool = v8 = false
    if v11 then
        failwith<unit> v10
method0()

assert_eq / actual: 999999999999999999L / expected: 999999999999999999L


## new_guid_from_date_time

In [ ]:
inl new_guid_from_date_time (date_time : date_time) =
    inl guid = guid.new_raw_guid ()
    date_time_guid_from_date_time guid date_time

()



In [ ]:
//// test
//// print_code=false

utc_now ()
|> new_guid_from_date_time
|> date_time_from_guid
|> fun date_time => new_time_span date_time (utc_now ()) |> total_seconds |> i32
|> _assert_eq 0

assert_eq / actual: 0 / expected: 0


## new_guid_from_timestamp

In [ ]:
inl new_guid_from_timestamp (timestamp : timestamp) =
    inl guid = guid.new_raw_guid ()
    timestamp_guid_from_timestamp guid timestamp

()



In [ ]:
//// test

utc_now ()
|> ticks
|> new_guid_from_timestamp
|> timestamp_from_guid
|> fun (timestamp timestamp) => (timestamp - (utc_now () |> ticks |> fun (timestamp x) => x)) / 100000i64
|> _assert_eq 0i64

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : System.DateTime = System.DateTime.UtcNow
    let v1 : (System.DateTime -> int64) = _.Ticks
    let v2 : int64 = v1 v0
    let v3 : System.Guid = System.Guid.NewGuid ()
    let v4 : (System.Guid -> string) = _.ToString()
    let v5 : string = v4 v3
    let v6 : (int64 -> string) = _.ToString()
    let v7 : string = v6 v2
    let v8 : string = v7.PadLeft (18, '0')
    let v9 : System.Guid = System.Guid $"{v8.[0..7]}-{v8.[8..11]}-{v8.[12..15]}-{v8.[16..17]}{v5.[21..]}"
    let v10 : (System.Guid -> string) = _.ToString()
    let v11 : string = v10 v9
    let v12 : int64 = int64 $"{v11.[0..7]}{v11.[9..12]}{v11.[14..17]}{v11.[19..20]}"
    let v13 : System.DateTime = System.DateTime.UtcNow
    let v14 : (System.DateTime -> int64) = _.Ticks
    let v15 : int64 = v14 v13
    let v16 : int64 = v12 - v15
    let v17 : int64 = v16 / 100000L
    let v18 : bool = v17 = 0L
    let v20 : bool =
        if v18 then
     

## main

In [ ]:
inl main () =
    types ()
    $"let date_time_guid_from_date_time x = !date_time_guid_from_date_time x" : ()
    $"let date_time_from_guid x = !date_time_from_guid x" : ()
    $"let timestamp_guid_from_timestamp x = !timestamp_guid_from_timestamp x" : ()
    $"let timestamp_from_guid x = !timestamp_from_guid x" : ()
    $"let new_guid_from_date_time x = !new_guid_from_date_time x" : ()
    $"let new_guid_from_timestamp x = !new_guid_from_timestamp x" : ()
    $"let format x = !format x" : ()
    $"let format_iso8601 x = !format_iso8601 x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("_")>]
#endif
type Any = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type Actio